# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    top_word = vocab[0]
    vocab_to_int = {w: i for i, w in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    
    print("Length of text: {}".format(len(text)))
    print("Most frequent word: {}={}".format(top_word, counts[top_word]))
    print("ID of most frequent word: {}".format(vocab_to_int[top_word]))
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Length of text: 104
Most frequent word: moe_szyslak=5
ID of most frequent word: 0
Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    tokens = {
        '.'  : "Period",
        ','  : "Comma",
        '"'  : "Quotation_Mark",
        ';'  : "Semicolon",
        '!'  : "Exclamation_Mark",
        '?'  : "Question_Mark",
        '('  : "Left_Parentheses",
        ')'  : "Right_Parentheses",
        '--' : "Dash",
        '\n' : "Return"
    }
    return tokens

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

Length of text: 69100
Most frequent word: period=6025
ID of most frequent word: 0


# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    
    # print("get_inputs()")
    # print("------------")
    
    input = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return input, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
# The number of layers of LSTM cells.
lstm_layers = 1

# CHANGED: I'm passing lstm_layers to get_init_cell() to be able to control it more directly.
def get_init_cell(batch_size, rnn_size, num_lstm_layers=None):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    global lstm_layers
    
    # print("get_init_cell()")
    # print("---------------")
    # print("batch_size: {}".format(batch_size))
    # print("rnn_size: {}".format(rnn_size))
    # print("lstm_layers: {}".format(num_lstm_layers))
    
    if num_lstm_layers:
        lstm_layers = num_lstm_layers
    
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * lstm_layers)
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name="initial_state")
    
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    
    # print("get_embed()")
    # print("-----------")
    # print("Input data: {}".format(input_data))
    # print("Vocabulary size: {}".format(vocab_size))
    # print("Embedding dimensions: {}".format(embed_dim))
    
    # Create the weight matrix of the embedding layer.
    # The size of the matrix is the number of words times number of hidden units = [vocab_size x embed_dim].
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data) 
    
    # print("Embedding tensor: {}".format(embed))
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
# I've added the parameter initial_state which was missing before.
# No idea why we created it above, but are not using it here.
# Also see: https://discussions.udacity.com/t/we-created-an-initial-state-in-get-init-cell-but-i-didnt-see-its-passed-into-the-rnn/250443.
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    
    # print("build_rnn()")
    # print("-----------")
    # print("Inputs: {}".format(inputs))
    
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name="final_state")
    
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    
    # print("build_nn()")
    # print("----------")
    
    embedding_tensor = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embedding_tensor)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    
    # print("get_batches()")
    # print("-------------")
    # print("Length of input text: {}".format(len(int_text)))
    # print("batch_size: {}".format(batch_size))
    # print("seq_length: {}".format(seq_length))
    
    words_per_batch = batch_size * seq_length
    # print("words_per_batch: {}".format(words_per_batch))
    
    # Calculate the number of batches that we will feed to the network.
    num_batches = len(int_text) // words_per_batch
    print("num_batches: {}".format(num_batches))
    
    # Drop the words that don't fit into a batch.
    int_text = int_text[:num_batches * words_per_batch]
    # print("Length of input text: {}".format(len(int_text)))
    
    batches = np.zeros((num_batches, 2, batch_size, seq_length))
    for i in range(0, num_batches):
        for j in range(0, batch_size):
            start = i * seq_length + j * (num_batches * seq_length)
            end = start + seq_length
            # print("start, end = {}, {}".format(start, end))

            # Inputs.
            x = np.array(int_text[start:end])
            
            # Targets, shifted by 1.
            y = np.zeros_like(x)
            y[:-1] = x[1:]
            if end == len(int_text):
                y[-1] = int_text[0]
            else:
                y[-1] = int_text[end]
            
            batches[i, 0, j] = x
            batches[i, 1, j] = y
            
    return batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

num_batches: 7
Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [31]:
num_epochs = 100  # Number of Epochs
batch_size = 64  # Batch Size
rnn_size = 256  # RNN Size
embed_dim = 300  # Embedding Dimension Size
seq_length = 75  # Sequence Length
learning_rate = 0.01  # Learning Rate
show_every_n_batches = 8  # Show stats for every n number of batches

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [32]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [33]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(1, num_epochs + 1):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            # if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
            if batch_i == (len(batches) - 1) and epoch_i % 10 == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i + 1,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

num_batches: 14
Epoch  10 Batch   14/14   train_loss = 2.857
Epoch  20 Batch   14/14   train_loss = 1.605
Epoch  30 Batch   14/14   train_loss = 1.015
Epoch  40 Batch   14/14   train_loss = 0.553
Epoch  50 Batch   14/14   train_loss = 0.337
Epoch  60 Batch   14/14   train_loss = 0.181
Epoch  70 Batch   14/14   train_loss = 0.129
Epoch  80 Batch   14/14   train_loss = 0.052
Epoch  90 Batch   14/14   train_loss = 0.044
Epoch 100 Batch   14/14   train_loss = 0.036
Model Trained and Saved


In [34]:
# -------------------------------------------
# I'VE USED THIS TO FIND GOOD HYPERPARAMETERS
# -------------------------------------------
# from tensorflow.contrib import seq2seq

# num_epochs = 100          # Number of Epochs.
# show_every_n_batches = 8  # Show stats for every n number of batches.
                    
# for batch_size in [64, 128]:
#     for lstm_layers in [1]:
#         for rnn_size in [64, 128, 256]:
#             for embed_dim in [100, 200, 300]:
#                 for seq_length in [15, 25, 50, 75]:
#                     for learning_rate in [0.1, 0.01, 0.001, 0.0001]:
#                         print("batch_size={}, lstm_layers={}, rnn_size={}, embed_dim={}, seq_length={}, learning_rate={}, show_every_n_batches={}"
#                               .format(batch_size, lstm_layers, rnn_size, embed_dim, seq_length, learning_rate, show_every_n_batches))
# 
#                         batches = get_batches(int_text, batch_size, seq_length)
#                         
#                         train_graph = tf.Graph()
#                         with train_graph.as_default():
#                             vocab_size = len(int_to_vocab)
#                             input_text, targets, lr = get_inputs()
#                             input_data_shape = tf.shape(input_text)
#                             cell, initial_state = get_init_cell(input_data_shape[0], rnn_size, lstm_layers)
#                             logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)
# 
#                             # Probabilities for generating words
#                             probs = tf.nn.softmax(logits, name='probs')
# 
#                             # Loss function
#                             cost = seq2seq.sequence_loss(
#                                 logits,
#                                 targets,
#                                 tf.ones([input_data_shape[0], input_data_shape[1]]))
# 
#                             # Optimizer
#                             optimizer = tf.train.AdamOptimizer(lr)
# 
#                             # Gradient Clipping
#                             gradients = optimizer.compute_gradients(cost)
#                             capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
#                             train_op = optimizer.apply_gradients(capped_gradients)
# 
#                         with tf.Session(graph=train_graph) as sess:
#                             sess.run(tf.global_variables_initializer())
# 
#                             for epoch_i in range(1, num_epochs + 1):
#                                 state = sess.run(initial_state, {input_text: batches[0][0]})
# 
#                                 for batch_i, (x, y) in enumerate(batches):
#                                     feed = {
#                                         input_text: x,
#                                         targets: y,
#                                         initial_state: state,
#                                         lr: learning_rate}
#                                     train_loss, state, _ = sess.run([cost, final_state, train_op], feed)
# 
#                                     # Show every <show_every_n_batches> batches
#                                     # if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
#                                     if batch_i == (len(batches) - 1) and epoch_i % 10 == 0:
#                                         print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
#                                             epoch_i,
#                                             batch_i + 1,
#                                             len(batches),
#                                             train_loss))

## Results of hyperparameter tuning

| Train Loss         | Batch Size    | LSTM Layers   | RNN Size    | Embedding Dimensions | Sequence Length | Learning Rate |
| ------------------ | ------------- | ------------- | ----------- | -------------------- | --------------- | ------------- |
| train_loss = 4.190 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=15 | learning_rate=0.1
| train_loss = 1.395 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=15 | learning_rate=0.01
| train_loss = 3.022 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=15 | learning_rate=0.001
| train_loss = 5.396 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=15 | learning_rate=0.0001
| train_loss = 3.875 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=25 | learning_rate=0.1
| train_loss = 1.601 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=25 | learning_rate=0.01
| train_loss = 3.476 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=25 | learning_rate=0.001
| train_loss = 5.471 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=25 | learning_rate=0.0001
| train_loss = 3.348 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=50 | learning_rate=0.1
| train_loss = 1.905 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=50 | learning_rate=0.01
| train_loss = 3.979 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=50 | learning_rate=0.001
| train_loss = 5.835 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=50 | learning_rate=0.0001
| train_loss = 3.147 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=75 | learning_rate=0.1
| train_loss = 2.199 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=75 | learning_rate=0.01
| train_loss = 4.215 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=75 | learning_rate=0.001
| train_loss = 5.872 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=100 | seq_length=75 | learning_rate=0.0001
| train_loss = 4.401 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=15 | learning_rate=0.1
| train_loss = 1.555 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=15 | learning_rate=0.01
| train_loss = 2.802 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=15 | learning_rate=0.001
| train_loss = 5.237 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=15 | learning_rate=0.0001
| train_loss = 4.016 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=25 | learning_rate=0.1
| train_loss = 1.658 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=25 | learning_rate=0.01
| train_loss = 3.258 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=25 | learning_rate=0.001
| train_loss = 5.416 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=25 | learning_rate=0.0001
| train_loss = 3.440 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=50 | learning_rate=0.1
| train_loss = 1.701 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=50 | learning_rate=0.01
| train_loss = 3.847 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=50 | learning_rate=0.001
| train_loss = 5.762 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=50 | learning_rate=0.0001
| train_loss = 3.193 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=75 | learning_rate=0.1
| train_loss = 2.106 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=75 | learning_rate=0.01
| train_loss = 4.076 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=75 | learning_rate=0.001
| train_loss = 5.828 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=200 | seq_length=75 | learning_rate=0.0001
| train_loss = 4.474 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=15 | learning_rate=0.1
| train_loss = 1.652 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=15 | learning_rate=0.01
| train_loss = 2.713 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=15 | learning_rate=0.001
| train_loss = 5.167 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=15 | learning_rate=0.0001
| train_loss = 4.121 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=25 | learning_rate=0.1
| train_loss = 1.766 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=25 | learning_rate=0.01
| train_loss = 3.115 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=25 | learning_rate=0.001
| train_loss = 5.364 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=25 | learning_rate=0.0001
| train_loss = 3.486 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=50 | learning_rate=0.1
| train_loss = 1.821 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=50 | learning_rate=0.01
| train_loss = 3.673 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=50 | learning_rate=0.001
| train_loss = 5.723 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=50 | learning_rate=0.0001
| train_loss = 3.382 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=75 | learning_rate=0.1
| train_loss = 1.925 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=75 | learning_rate=0.01
| train_loss = 3.974 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=75 | learning_rate=0.001
| train_loss = 5.793 | batch_size=64 | lstm_layers=1 | rnn_size=64 | embed_dim=300 | seq_length=75 | learning_rate=0.0001
| train_loss = 4.161 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=15 | learning_rate=0.1
| train_loss = 0.665 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=15 | learning_rate=0.01
| train_loss = 2.042 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=15 | learning_rate=0.001
| train_loss = 5.005 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=15 | learning_rate=0.0001
| train_loss = 3.739 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=25 | learning_rate=0.1
| train_loss = 0.595 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=25 | learning_rate=0.01
| train_loss = 2.697 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=25 | learning_rate=0.001
| train_loss = 5.146 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=25 | learning_rate=0.0001
| train_loss = 3.028 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=50 | learning_rate=0.1
| train_loss = 0.788 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=50 | learning_rate=0.01
| train_loss = 3.320 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=50 | learning_rate=0.001
| train_loss = 5.553 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=50 | learning_rate=0.0001
| train_loss = 2.614 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=75 | learning_rate=0.1
| train_loss = 1.010 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=75 | learning_rate=0.01
| train_loss = 3.720 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=75 | learning_rate=0.001
| train_loss = 5.727 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=100 | seq_length=75 | learning_rate=0.0001
| train_loss = 4.412 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=15 | learning_rate=0.1
| train_loss = 0.616 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=15 | learning_rate=0.01
| train_loss = 1.722 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=15 | learning_rate=0.001
| train_loss = 4.741 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=15 | learning_rate=0.0001
| train_loss = 3.876 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=25 | learning_rate=0.1
| train_loss = 0.581 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=25 | learning_rate=0.01
| train_loss = 2.520 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=25 | learning_rate=0.001
| train_loss = 4.944 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=25 | learning_rate=0.0001
| train_loss = 3.104 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=50 | learning_rate=0.1
| train_loss = 0.611 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=50 | learning_rate=0.01
| train_loss = 3.123 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=50 | learning_rate=0.001
| train_loss = 5.389 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=50 | learning_rate=0.0001
| train_loss = 3.043 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=75 | learning_rate=0.1
| train_loss = 0.811 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=75 | learning_rate=0.01
| train_loss = 3.518 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=75 | learning_rate=0.001
| train_loss = 5.563 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=200 | seq_length=75 | learning_rate=0.0001
| train_loss = 4.491 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=15 | learning_rate=0.1
| train_loss = 0.706 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=15 | learning_rate=0.01
| train_loss = 1.619 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=15 | learning_rate=0.001
| train_loss = 4.625 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=15 | learning_rate=0.0001
| train_loss = 4.090 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=25 | learning_rate=0.1
| train_loss = 0.583 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=25 | learning_rate=0.01
| train_loss = 2.341 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=25 | learning_rate=0.001
| train_loss = 4.847 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=25 | learning_rate=0.0001
| train_loss = 3.219 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=50 | learning_rate=0.1
| train_loss = 0.534 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=50 | learning_rate=0.01
| train_loss = 3.000 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=50 | learning_rate=0.001
| train_loss = 5.284 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=50 | learning_rate=0.0001
| train_loss = 2.824 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=75 | learning_rate=0.1
| train_loss = 0.715 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=75 | learning_rate=0.01
| train_loss = 3.332 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=75 | learning_rate=0.001
| train_loss = 5.508 | batch_size=64 | lstm_layers=1 | rnn_size=128 | embed_dim=300 | seq_length=75 | learning_rate=0.0001
| train_loss = 0.271 | batch_size=64 | lstm_layers=1 | rnn_size=256 | embed_dim=100 | seq_length=15 | learning_rate=0.01
| train_loss = 0.139 | batch_size=64 | lstm_layers=1 | rnn_size=256 | embed_dim=100 | seq_length=25 | learning_rate=0.01
| train_loss = 0.075 | batch_size=64 | lstm_layers=1 | rnn_size=256 | embed_dim=100 | seq_length=50 | learning_rate=0.01
| train_loss = 0.089 | batch_size=64 | lstm_layers=1 | rnn_size=256 | embed_dim=100 | seq_length=75 | learning_rate=0.01
| train_loss = 0.868 | batch_size=64 | lstm_layers=1 | rnn_size=256 | embed_dim=200 | seq_length=15 | learning_rate=0.01
| train_loss = 0.135 | batch_size=64 | lstm_layers=1 | rnn_size=256 | embed_dim=200 | seq_length=25 | learning_rate=0.01
| train_loss = 0.053 | batch_size=64 | lstm_layers=1 | rnn_size=256 | embed_dim=200 | seq_length=50 | learning_rate=0.01
| train_loss = 0.040 | batch_size=64 | lstm_layers=1 | rnn_size=256 | embed_dim=200 | seq_length=75 | learning_rate=0.01
| train_loss = 0.262 | batch_size=64 | lstm_layers=1 | rnn_size=256 | embed_dim=300 | seq_length=15 | learning_rate=0.01
| train_loss = 0.133 | batch_size=64 | lstm_layers=1 | rnn_size=256 | embed_dim=300 | seq_length=25 | learning_rate=0.01
| train_loss = 0.051 | batch_size=64 | lstm_layers=1 | rnn_size=256 | embed_dim=300 | seq_length=50 | learning_rate=0.01
| **train_loss = 0.038** | batch_size=64 | lstm_layers=1 | rnn_size=256 | embed_dim=300 | seq_length=75 | learning_rate=0.01

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [35]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [36]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [37]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    input0 = loaded_graph.get_tensor_by_name("input:0")
    initial_state0 = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state0 = loaded_graph.get_tensor_by_name("final_state:0")
    probs0 = loaded_graph.get_tensor_by_name("probs:0")
    return input0, initial_state0, final_state0, probs0


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [41]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    words = list(int_to_vocab.values())
    return np.random.choice(a=words, p=probabilities)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [42]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: so lenny, let's say you pull a thorn out of the pope's butt and he grants ya one wish. what'll it be?
lenny_leonard: hmm, only one, huh... well, i've always wondered what it would feel like goin' from in this?
carl_carlson: hey at this channel?
homer_simpson: well, i did. the springfield police have to call me, and i invented it.


moe_szyslak:(laughs) i believe in some kind of polygon.
carl_carlson: hexa- or octa-?
lenny_leonard: only one way to decide: arm wrestling.
moe_szyslak:(sadly muttering to my beer. no pal,(loud) achem!
homer_simpson:(loud whisper) let's try(loud) hey, moe, can you think of a way to please a woman that starts with" f"?
moe_szyslak: hey, you know, i'm back where i belong, eh?
smile:(low baritone voice) you're not alone... babar, king of the elephants. i know your money's no beer.


## Analysis
My first network overfitted the data. It achieved a loss of 0.036 and produced almost word-by-word copies of the original dataset. According to the review, the sequence length should be about the average number of words in a sentence (between 15 and 20). I assume that the high sequence length that I used contributed quite a bit to the overfitted model. Here are the parameters that I used:
* num_epochs = 100
* batch_size = 64
* rnn_size = 256
* embed_dim = 300
* seq_length = 75
* learning_rate = 0.01

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.